# Exercise 2: Indexing and Probabilistic Retrieval
The following code is modified from Exercise 1. It is used to construct the vocabulary and vectorize the documents and query.

In [7]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')



stemmer = PorterStemmer()

# Tokenize, stem a document
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

# Read a list of documents from a file. Each line in a file is a document
with open("bread.txt") as f:
# with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]

# create the vocabulary
vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()

# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        idf[term] = math.log(num_documents/sum(term in document for document in documents), math.e)
    return idf

# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]/max_count
    return vector

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy/math.sqrt(sumxx*sumyy)
    return result

def vectorize_query(query, vocabulary, idf):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    return query_vector
    
def search_vec(query, k):
    query_vector = vectorize_query(query, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    ans = []
    indices = []
    for i in range(min(k,len(original_documents))):
        ans.append(original_documents[scores[i][1]])
        indices.append(scores[i][1])
    return ans, indices, query_vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

[nltk_data] Downloading package stopwords to /home/yawen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/yawen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Question 1 -  Distributed Information Retrieval

In this exercise we implement a core process of Fagin's algorithm, the parallel scanning of the posting lists. Assume an aggregation function that returns the sum of the tf-idf scores given the terms in a document.

We assume that the posting lists for each term of a retrieval system are running on a different node.

We first create a dictionary $h$, where each entry of the dictionary corresponds to a posting list for a term, assumed to be hosted in a different node. Explore the structure of $h$, to understand it. We suggest to first use the simple collection breads.txt.

In [8]:
import numpy as np
import operator

doc_vecs = np.transpose(np.array(document_vectors))
h = {}
for i, term in enumerate(vocabulary):
    ha = {}
    for docj in range(len(original_documents)):
        tfidf = doc_vecs[i][docj]
        ha[docj] = tfidf
    sorted_ha = sorted(ha.items(), key=operator.itemgetter(1), reverse=True)
    h[term] = sorted_ha


In [9]:
h

{'art': [(2, 1.6094379124341003), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0)],
 'bake': [(0, 0.9162907318741551),
  (3, 0.9162907318741551),
  (1, 0.0),
  (2, 0.0),
  (4, 0.0)],
 'best': [(1, 0.45814536593707755),
  (4, 0.45814536593707755),
  (0, 0.0),
  (2, 0.0),
  (3, 0.0)],
 'book': [(4, 0.8047189562170501), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)],
 'bread': [(3, 0.9162907318741551),
  (0, 0.45814536593707755),
  (1, 0.0),
  (2, 0.0),
  (4, 0.0)],
 'cake': [(3, 1.6094379124341003), (0, 0.0), (1, 0.0), (2, 0.0), (4, 0.0)],
 'comput': [(2, 1.6094379124341003), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0)],
 'french': [(4, 0.8047189562170501), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)],
 'london': [(1, 0.8047189562170501), (0, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)],
 'numer': [(2, 1.6094379124341003), (0, 0.0), (1, 0.0), (3, 0.0), (4, 0.0)],
 'pastri': [(3, 0.9162907318741551),
  (4, 0.9162907318741551),
  (0, 0.0),
  (1, 0.0),
  (2, 0.0)],
 'pie': [(1, 0.9162907318741551),
  (3, 0.9162907318741551),

### Complete the following function that implements the Fagin algorithm.

In [10]:
sorted_ha

[(0, 0.8047189562170501), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]

In [11]:
def fagin_algorithm(query, h, k, vocabulary):
        
    # Split and stem the query
    q = query.split()
    q = set([stemmer.stem(w) for w in q])
    query_term_cnt = len(q)
    
    print("q: {}".format(q))
    
    # select the posting lists for the query terms
    posting_lists = {}
    for term in q:
        if term in h:
            posting_lists[term] = h[term]
    
    print("Posting lists: {}".format(posting_lists))
    
    max_len = len(documents)
    
    # Traverse the selected posting lists until we found k documents that appear in ALL posting lists
    # This corresponds to phase 1 of Fagin's algorithm.
    # As a result you produce a dictionary documents_occurrences, with the document identifiers as keys, 
    # and the number of documents found as value.
    # We stop traversing the posting lists until we have found k documents that appear in ALL posting lists 
    # of the query terms
        
    documents_occurrences = dict()

    # YOUR CODE HERE
    # go through the filtered posting lists
    l = 0
    found_documents = 0
    while l < max_len:
        for term in q:
            # Get the current doc for each query term (first doc for q1, q2, q3...)
            doc = posting_lists[term][l][0]
            
            if doc in documents_occurrences.keys():
                documents_occurrences[doc] = documents_occurrences[doc] + 1
            else:
                documents_occurrences[doc] = 1
            
            # check if all posting list has this doc
            if documents_occurrences[doc] == query_term_cnt:
                found_documents = found_documents + 1
            
            # check if we found k docs
            if found_documents == k:
                l = max_len + 1
                break
        l = l + 1
                
    print("Document occurrences: {}".format(documents_occurrences))
        
    # Retrieve for the found documents the tfidf values and add them up.
    # For simplicity, we do not distinguish the cases whether the values have already been retrieved in the 
    # previous phase, or whether we use random access to obtain those values
    
    tfidf = {}
    for d in documents_occurrences.keys():
        t = 0
        for term in q:
            t = t + dict(posting_lists[term])[d]
        tfidf[d] = t
        
    # Finally we compute the top-k documents and return the answer
    
    ans = sorted(tfidf.items(), key=lambda x:x[1], reverse = True)[:k]
    return ans


ans = fagin_algorithm("bread recipe", h, 2, vocabulary)
print("\n", ans)
for document_id in ans:
    print(original_documents[document_id[0]])

q: {'bread', 'recip'}
Posting lists: {'bread': [(3, 0.9162907318741551), (0, 0.45814536593707755), (1, 0.0), (2, 0.0), (4, 0.0)], 'recip': [(2, 0.22314355131420976), (3, 0.22314355131420976), (0, 0.11157177565710488), (4, 0.11157177565710488), (1, 0.0)]}
Document occurrences: {0: 2, 1: 1, 2: 1, 3: 2}

 [(3, 1.1394342831883648), (0, 0.5697171415941824)]
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
How to Bake Breads Without Baking Recipes


Produce your own datasets to explore the behavior of the algorithm. Create a dataset such that for retrieving a 2 term query a total of 6 documents needs to be retrieved in the first phase of the algorithm (as in the example in the lecture).

## Question 2: Probabilistic Retrieval

Following the notation used in class, let us denote the set of terms by $T=\{k_i|i=1,...,m\}$, the set of documents by $D=\{d_j |j=1,...,n\}$, and let $d_i=(w_{1j},w_{2j},...,w_{mj})$. We are also given a query  $q=(w_{1q},w_{2q},...,w_{mq})$. In the lecture we studied that, 

$sim(q,d_j) = \sum^m_{i=1} \frac{w_{ij}}{|d_j|}\frac{w_{iq}}{|q|}$ .  (1)

Another way of looking at the information retrieval problem is using a probabilistic approach. The probabilistic view of information retrieval consists of determining the conditional probability $P(q|d_j)$ that for a given document $d_j$ the query by the user is $q$. So, practically in probabilistic retrieval when a query $q$ is given, for each document it is evaluated how probable it is that the query is indeed relevant for the document, which results in a ranking of the documents.

In order to relate vector space retrieval to a probabilistic view of information retrieval, we interpret the weights in Equation (1) as follows:

-  $w_{ij}/|d_j|$ can be interpreted as the conditional probability $P(k_i|d_j)$ that for a given document $d_j$ the term $k_i$ is important (to characterize the document $d_j$).

-  $w_{iq}/|q|$ can be interpreted as the conditional probability $P(q|k_i)$ that for a given term $k_i$ the query posed by the user is $q$. Intuitively, $P(q|k_i)$ gives the amount of importance given to a particular term while querying.

With this interpretation you can rewrite Equation (1) as follows:

$sim(q,d_j) = \sum^m_{i=1} P(k_i|d_j)P(q|k_i)$ (2)

### Question a
Show that indeed with the probabilistic interpretation of weights of vector space retrieval, as given in Equation (2), the similarity computation in vector space retrieval results exactly in the probabilistic interpretation of information retrieval, i.e., $sim(q,d_j)= P(q|d_j)$.
Given that $d_j$ and $q$ are conditionally independent, i.e., $P(d_j \cap q|ki) = P(d_j|k_i)P(q|k_i)$. You can assume existence of joint probability density functions wherever required. (Hint: You might need to use Bayes theorem)

### Question b
Using the expression derived for $P(q|d_j)$ in (a), obtain a ranking (documents sorted in descending order of their scores) for the documents $P(k_i|d_1) = (0, 1/3, 2/3)$, $P(k_i|d_2) =(1/3, 2/3, 0)$, $P(k_i|d_3) = (1/2, 0, 1/2)$, and $P (k_i|d_4) = (3/4, 1/4, 0)$ and the query $P(q|k_i) = (1/5, 0, 2/3)$.

### Question c

 Note that the model described in a. provides a probabilistic interpretation for vector space retrieval where weights are interpreted as probabilities . Compare to the probabilistic retrieval model based on language models introduced in the lecture and discuss the differences.

## Question 3 - Inverted Files

We learnt in the lecture that terms are typically stored in an inverted list. Now, in the inverted list, instead of only storing document identifiers of the documents in which the term appears, assume we also store an *offset* of the appearance of a term in a document. An $offset$ of a term $l_k$ given a document is defined as the number of words between the start of the document and $l_k$. Thus our inverted list is now:

$l_k= \langle f_k: \{d_{i_1} \rightarrow [o_1,\ldots,o_{n_{i_1}}]\}, 
\{d_{i_2} \rightarrow [o_1,\ldots,o_{n_{i_2}}]\}, \ldots, 
\{d_{i_k} \rightarrow [o_1,\ldots,o_{n_{i_k}}]\} \rangle$

This means that in document $d_{i_1}$ term $l_k$ appears $n_{i_1}$ times and at offset $[o_1,,o_{n_{i_1}}]$, where $[o_1,,o_{n_{i_1}}]$ are sorted in ascending order, these type of indices are also known as term-offset indices. An example of a term-offset index is as follows:

**Obama** = $⟨4 : {1 → [3]},{2 → [6]},{3 → [2,17]},{4 → [1]}⟩$

**Governor** = $⟨2 : {4 → [3]}, {7 → [14]}⟩$

**Election** = $⟨4 : {1 → [1]},{2 → [1,21]},{3 → [3]},{5 → [16,22,51]}⟩$

Which is to say that the term **Governor** appear in 2 documents. In document 4 at offset 3, in document 7 at offset 14. Now let us consider the *SLOP/x* operator in text retrieval. This operator has the syntax: *QueryTerm1 SLOP/x QueryTerm2* finds occurrences of *QueryTerm1* within $x$ (but not necessarily in that order) words of *QueryTerm2*, where $x$ is a positive integer argument ($x \geq 1$). Thus $x = 1$ demands that *QueryTerm1* be adjacent to *QueryTerm2*.

1. List the set of documents that satisfy the query **Obama** *SLOP/2* **Election**.
2. List each set of values for which the query **Obama** *SLOP/x* **Election** has a different set of documents as answers (starting from $x = 1$). 
3. Consider the general procedure for "merging" two term-offset inverted lists for a given document, to determine where the document satisfies a *SLOP/x* clause (since in general there will be many offsets at which each term occurs in a document). Let $L$ denote the total number of occurrences of the two terms in the document. Assume we have a pointer to the list of occurrences of each term and can move the pointer along this list. As we do so we check whether we have a hit for $SLOP/x$ (i.e. the $SLOP/x$ clause is satisfied). Each move of either pointer counts as a step. Based on this assumption is there a general "merging" procedure to determine whether the document satisfies a $SLOP/x$ clause, for which the following is true? Justify your answer.

    1. The merge can be accomplished in a number of steps linear in $L$ regardless of $x$, and we can ensure that each pointer moves only to the right (i.e. forward).
    2. The merge can be accomplished in a number of steps linear in $L$, but a pointer may be forced to move to the left (i.e. backwards).
    3. The merge can require $x \times L$ steps in some cases.

## Question 4: Create inverted files using MapReduce
In this exercise, we want to create the inverted files using the MapReduce framework. Write the **map** and **reduce** function. Note that the code of **run_mapreduce** is just a simulation of the behavior of  map-reduce system.

In [12]:
with open("epfldocs.txt") as f:
# with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]
docs_n_doc_ids = list(zip(range(len(documents)), documents))

In [13]:
# Take a pair (doc_id, content). Example: (0, ['how', 'to', 'bake', 'bread', 'without', 'bake', 'recip'])
# Return: list of (word, doc_id). Example: ('bake', 0)
# For simplicity, we assume each mapper handles a document
def map_function(doc_id, content):
    keyvals = []
    for word in content:
        keyvals.append((word, doc_id))
    return keyvals

In [14]:
# Take a list of pairs (word, doc_id). Example:  [('bake', 0), ('bake', 0), ('bake', 3)]
# Return: (word, frequency, list of document ids). Example ('bake', 3, [0, 0, 3])
# For simplicity, we assume each reducer handles a word
def reduce_function(lst):
    total = 0
    word = lst[0][0]
    doc_ids = []
    for w, doc_id in lst:
        assert(word==w) # Assume each reducer handles 1 word
        total = total + 1
        doc_ids.append(doc_id)
    return (word, total, doc_ids)

In [15]:
# We simulate the mapreduce framework by this function
def run_mapreduce(docs_n_doc_ids):
    # Map phase
    key_values = []
    for doc_id, doc_content in docs_n_doc_ids:
        key_values.extend(map_function(doc_id, doc_content))
    # Shuffle phase
    values = set(map(lambda x:x[0], key_values))
    grouped_key_values = [[y for y in key_values if y[0]==x] for x in values]
    # Reduce phase
    inverted_files = []
    for grouped_key_value in grouped_key_values:
        word, total, doc_ids = reduce_function(grouped_key_value)
        inverted_files.append((word, total, doc_ids))
    return inverted_files

In [16]:
run_mapreduce(docs_n_doc_ids)

[('rais', 2, [499, 1014]),
 ('spectra', 1, [661]),
 ('httpstcoyz9pvhlfdq', 1, [343]),
 ('progenitor', 1, [734]),
 ('détermin', 1, [1031]),
 ('preventiteu', 1, [588]),
 ('point', 7, [108, 175, 251, 272, 292, 312, 1067]),
 ('nice', 7, [22, 100, 234, 372, 591, 808, 1036]),
 ('prépare', 1, [265]),
 ('httpstco3wjjjpb40a', 1, [947]),
 ('nation', 3, [54, 64, 480]),
 ('ieeeasm', 1, [407]),
 ('httpstcowt09cxhcaa', 1, [1050]),
 ('light',
  13,
  [36, 44, 328, 406, 527, 544, 562, 597, 597, 714, 757, 831, 908]),
 ('cost', 1, [595]),
 ('knowledg', 1, [524]),
 ('voic', 1, [65]),
 ('distinguish', 1, [293]),
 ('pierr', 2, [741, 837]),
 ('researchcheck', 1, [296]),
 ('antivir', 1, [577]),
 ('httpstcogjfgwn1gcq', 1, [197]),
 ('slasher', 1, [702]),
 ('acquisit', 1, [80]),
 ('oscienc', 9, [24, 67, 111, 153, 328, 355, 527, 811, 932]),
 ('dispositif', 1, [383]),
 ('open',
  26,
  [2,
   49,
   54,
   109,
   188,
   245,
   260,
   348,
   393,
   393,
   420,
   442,
   453,
   480,
   504,
   515,
   587,